## Modelado de datos para un sistema transaccional en línea a escala de Internet mediante Amazon DynamoDB

La tecnología de sistemas de gestión de bases de datos relacionales (RDBMS) fue la opción preferida para el almacenamiento de datos durante décadas. En los últimos años, el auge de las aplicaciones a escala de Internet y el crecimiento exponencial de los datos han desafiado la forma en que las industrias han diseñado sus estrategias de bases de datos. Las organizaciones están viendo claros beneficios en escalabilidad y rendimiento al migrar sistemas transaccionales en línea de misión crítica construidos sobre RDBMS a tecnologías NoSQL.

En esta publicación, analizamos las ventajas de diseñar un sistema online altamente transaccional en Amazon DynamoDB, una base de datos NoSQL de valor clave, sin servidor y totalmente administrada. Utilizando una aplicación IMDb de muestra, examinamos el diseño de tabla única, un enfoque de modelado de datos común para conjuntos de datos relacionales en DynamoDB.

Las tecnologías RDBMS tienen sus limitaciones cuando se trata de soportar sistemas transaccionales en línea a gran escala. Su aplicación estará sujeta a un esquema estricto, y cualquier iteración puede requerir una planificación significativa. Una aplicación evoluciona con el tiempo, al igual que su esquema. La creación de una estrategia de datos agnóstica ayuda a minimizar las interrupciones del esquema causadas por la evolución de sus requisitos empresariales. Dado que DynamoDB es una base de datos NoSQL de valores clave, los cambios de esquema tienen un impacto mínimo en el modelo de datos de DynamoDB, siempre que el patrón de acceso permanezca inalterado. DynamoDB es altamente escalable y admite una latencia constante de milisegundos de un solo dígito a cualquier escala. Mediante el uso de funciones empresariales, como las transacciones ACID, DynamoDB Accelerator (DAX) y las tablas globales, las organizaciones pueden beneficiarse de un mejor rendimiento y disponibilidad de las aplicaciones a escala de Internet.

### Diseño de tablas de DynamoDB para aplicaciones a escala de Internet

Una base de datos relacional es una colección de entidades empresariales heterogéneas que se almacenan en varias tablas y se relacionan entre sí mediante claves externas. Durante el acceso, las consultas SQL pueden unir conjuntos de datos de varias tablas y responder a la aplicación cliente. Con el crecimiento de los conjuntos de datos, resulta difícil obtener un rendimiento constante de las bases de datos relacionales. DynamoDB está diseñada para proporcionar un rendimiento de consulta consistente a cualquier escala, siempre que las tablas de destino se diseñen lo más cerca posible del patrón de acceso. La recomendación es que los datos que se consultan juntos permanezcan juntos. En este enfoque de diseño, las entidades empresariales heterogéneas se almacenan como una colección de elementos en una única tabla de DynamoDB. Esto reduce las uniones de consulta y los viajes de ida y vuelta por la red, lo que se traduce en un rendimiento de consulta más rápido y coherente.

Cada elemento almacenado en una tabla de DynamoDB tiene una clave de partición obligatoria y una clave de ordenación opcional, denominadas conjuntamente clave principal. En un enfoque de diseño de tabla única, debe elegir cuidadosamente los atributos de clave de partición y clave de ordenación, ya que pueden mejorar significativamente la selectividad (rendimiento de la consulta) de los elementos recuperados por una consulta. Elija una clave de partición de cardinalidad alta para garantizar una distribución eficaz de los datos entre las particiones. Al elegir la clave de ordenación, puede utilizar técnicas como las claves de ordenación compuestas para definir una relación jerárquica en los datos, de forma que pueda consultar cualquier nivel de la jerarquía. Esto permitirá a la aplicación resolver patrones de acceso mediante consultas y evitar escaneos de tablas. Los escaneos de tabla consumen muchos recursos, pueden causar un rendimiento más lento y son costosos debido al mayor consumo de la unidad de capacidad de lectura (RCU) de la tabla. En este post, repasamos un conjunto de datos IMDb de ejemplo para explicar el diseño de una tabla única y demostrar cómo se puede utilizar una clave de ordenación compuesta para resolver múltiples patrones de acceso.

### Explorar el modelo de datos de IMDb

La base de datos MySQL tiene datos en una estructura relacional normalizada en seis tablas. El esquema de las tablas se adhiere al esquema y las relaciones de los archivos cargados en la tabla. La Figura 9 describe el modelo entidad-relación entre el reparto y el equipo de las películas.

![image.png](https://d2908q01vomqb2.cloudfront.net/887309d048beef83ad3eabf2a79a64a389ab1c9f/2023/02/10/Picture9.png)

Las tablas de la base de datos imdbdata que se muestran en la Figura 9 son:

* title_basics almacena las películas publicadas en Estados Unidos después del año 2000. tconst es una clave alfanumérica asignada de forma exclusiva a cada película.
* title_akas almacena las regiones publicadas, los idiomas y los respectivos títulos de las películas. Tiene una relación de uno a muchos con title_basics.title_ratings almacena las calificaciones de las películas y el recuento de votos. Para este ejercicio, puede suponer que la información se actualiza con alta frecuencia tras el estreno de la película. Tiene una relación de uno a uno con title_basics.
* title_principals almacena información sobre el reparto y el equipo. Tiene una relación de uno a muchos con title_basics.
* title_crew almacena información sobre el guionista y el director. Tiene una relación de uno a uno con title_basics.
* name_basics almacena información sobre el reparto y el equipo. Cada entrada tiene asignado un valor nconst único. Tiene una relación de uno a muchos con title_crew y title_principals.

### Patrones de acceso a DynamoDB para la aplicación IMDb

DynamoDB es la mejor opción para soportar una respuesta de baja latencia para aplicaciones a escala de Internet. Recuerde que los datos que se consultan juntos deben permanecer juntos. Este enfoque ayuda a soportar una latencia de consulta de un milisegundo a cualquier escala y proporciona un rendimiento constante para aplicaciones a escala de Internet como IMDb. Para obtener más información, consulte las directrices de prácticas recomendadas para el diseño de soluciones en DynamoDB. Identificamos seis patrones de acceso para la aplicación IMDb para ilustrar el diseño de tablas DynamoDB.

* Buscar todo el reparto y el equipo de una película.
* Buscar detalles de una película (título, año de publicación, duración, géneros, directores, guionistas y otra información).
* Buscar la clasificación IMDb de una película.
* Buscar todas las regiones, idiomas y títulos publicados de una película.
* Buscar el título original de una película publicada en varios idiomas y regiones.
* Buscar todas las películas de un actor entre los años 2002 y 2010.

### Diseño de tabla única de DynamoDB para la aplicación IMDb
A continuación, repasaremos el diseño de la tabla única y la clave compuesta para la aplicación IMDb. Anteriormente, hemos comentado que cada película está asociada a un identificador único: tconst. Los cinco primeros patrones de acceso de la aplicación se basan en las películas. Utilizar tconst como clave de partición ayuda a distribuir los datos entre varias particiones y a soportar los patrones de acceso. En este ejemplo, utilizamos una técnica de clave de ordenación compuesta para almacenar y acceder a la información relacionada. El atributo de clave de ordenación se prefija con caracteres constantes conocidos por las aplicaciones. El prefijo agrupa uno o más elementos relacionados como una única colección. Las convenciones de nomenclatura para el prefijo se definen de forma personalizada, por lo que no dude en utilizar cualquiera según los estándares de su aplicación. A continuación se ofrece una lista detallada de los prefijos utilizados para el diseño de tablas de destino:

* DETL prefijo colección de elementos: title_basics contiene metadatos de la película y title_principals tiene toda la información sobre el reparto y el equipo. La información de ambas tablas es estática una vez que se estrena una película. Los patrones de acceso requieren que las películas y la información sobre su reparto y personal se obtengan juntas. Los detalles de la película y la información asociada sobre el reparto y el equipo se desnormalizan como un único elemento de la tabla. Esto ayudará a acceder a la información con una sola consulta.

* Colección de elementos con prefijo REGN: title_akas contiene todas las regiones, idiomas y títulos publicados de una película. La información cambia con poca frecuencia y tiene una relación de uno a muchos con title_basics. Estos registros se clasifican utilizando REGN como prefijo y se modelan como uno a uno en la tabla DynamoDB.

* Colección de elementos con prefijo RTGN: Cada película recibe valoraciones basadas en los votos de los usuarios de IMDb. Cada vez que se recibe una nueva votación, la información de title_ratings se actualiza con la última calificación y el recuento de votos. Se trata de información dinámica que cambia con frecuencia. Para minimizar la capacidad de lectura y escritura utilizada durante las frecuentes operaciones de actualización, la información de calificación se mantiene fuera de los registros prefijados DETL.
Utilizando una clave de ordenación compuesta, se puede crear una colección de entidades heterogéneas y almacenarlas en una única tabla. La Figura 10 muestra el tconst como mpkey y la clave de clasificación compuesta dentro de los atributos mskey en una única tabla DynamoDB.

![image](https://d2908q01vomqb2.cloudfront.net/887309d048beef83ad3eabf2a79a64a389ab1c9f/2023/02/10/Picture10.png)

Cada película tiene su propio reparto y equipo, que incluye un productor, un director, actores y otros miembros del equipo. Se utiliza un identificador único, nconst, para identificar a cada miembro del reparto y del equipo. El último patrón de acceso de la lista es la selección de todas las películas de un actor dentro de un rango de años determinado. Este patrón de acceso no puede resolverse mediante consulta porque nconst no forma parte de la clave de partición de la tabla base. Para resolver este patrón de acceso, puede seguir cualquiera de los enfoques que se mencionan a continuación:

1. Utilice la técnica de lista de adyacencia para manejar las relaciones de muchos a muchos entre actores y película. Ambas entidades tienen identificadores únicos y no se solapan en valores. Este método requiere lógica de procesamiento adicional durante la ingesta para tratar la sobrecarga de claves de partición.

2. Cree un índice secundario global (GSI) en el atributo nconst de la tabla base y ejecute la consulta en el GSI.

Dado que el patrón de acceso sólo requiere una lista de películas de un actor dentro de un año específico, sólo necesita crear un GSI en la tabla base. La figura siguiente representa un modelo GSI con nconst como clave de partición y year como clave de ordenación para permitir consultas de rango.

![image](https://d2908q01vomqb2.cloudfront.net/887309d048beef83ad3eabf2a79a64a389ab1c9f/2023/02/10/Picture11.png)

![image](https://d2908q01vomqb2.cloudfront.net/887309d048beef83ad3eabf2a79a64a389ab1c9f/2023/02/14/migration35.png)